In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inflection
import dill

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, roc_auc_score

SEED = 0

def load_dataset(path):
    """
    Return the dataset in pandas dataframe format with lowercase columns
    and each column name seperated by underscore.
    
    Args:
    * path, str: file directory or file link of dataset
    
    Output:
    * pd.DataFrame
    
    Notes:
    * This function is used to read a parquet format.
      If the format is not parquet, change the first line of code.
      i.e. pd.read_csv.
    
    """
    df = pd.read_csv(path)
    df.columns = [inflection.underscore(var) for var in list(df.columns)]    
    return df

def parse_to_numeric(df, var='total_charges'):
    """
    Parse the numerical variables that are still in object dtype. 
    
    Args:
    * df, pd.DataFrame: the dataset
    * var (default: 'total_charges'), str: a variable to be parsed
    * fillna (default: True), bool: option to fill missing values or not
    Return:
    * pd.DataFrame
    
    """
    df = df.copy()
    df[var] = pd.to_numeric(df[var], errors='coerce')
    return df

def split_dataset(df, target='churn', test_size=0.2, seed=SEED):
    """
    Return the train and validation/test set with specified split ratio.
    
    Args:
    * df, pd.DataFrame: the dataset to be splitted
    * target (default: 'churn'), str: the target variable
    * test_size (default: 0.2), float: the ratio of test size after splitting
    * seed (default: SEED), int: random number for reproducibility
    
    Output:
    * train, val (optional: test), pd.DataFrame: training and validation/test sets
    
    Notes:
    * The purpose of splitted is to avoid data leakage and
      to make an hold-out dataset for testing the model performance.
    * Use strafied sampling to make training and validation/test data 
      have similar distribution.

    """
    return train_test_split(
        df,
        test_size=test_size,
        random_state=seed,
        stratify=df[target]
    )

def dataset_summary(df):
    """
    Return the following information from dataset:
    variable name, number of unique value, pandas dtype, 
    number of missing values, percentage of missing values, 
    and list of unique values.
    
    Args:
    * df, pd.DataFrame: the dataset
    
    Output:
    * table, pd.DataFrame
    """
    table = pd.DataFrame(
                columns=['variable',
                         'no_unique',
                         'pandas_dtype',
                         'missing_value',
                         '%_missing_values',
                         'unique_value'
                ]
    )

    for i, var in enumerate(df.columns):
        table.loc[i] = [var,
                        df[var].nunique(),
                        df[var].dtypes,
                        df[var].isnull().sum(),
                        df[var].isnull().sum() * 100 / df.shape[0],
                        df[var].unique().tolist()
        ]
    return table

def train_the_model(train, val, target='churn', seed=SEED):
    """
    Return the trained model and print the chosen evaluatoin metric.
    
    Args:
    * train, pd.DataFrame: training set
    * val, pd.DataFrame: either validation or test set
    * target (default: 'churn'), str: the target variable
    * seed (default: SEED), int: random number for reproducibility
    
    Output:
    * model, .bin: the trained model
    
    Notes:
    * Do not forget to make features and target matrix, X and y.
    * If using DictVectorizer, convert features matrix into dictionary.
    * Use pipeline instead of manual data preprocessing and modelling.
    * Use Logistic Regression to train the model for building a simple
      model.
      
    """
    X_train = train.drop(target, axis=1)
    X_val = val.drop(target, axis=1)
    
    # use FunctionTransformer to automatically convert features matrix to dictionary
    #def matrix_to_dict(X): return X.to_dict(orient='records')
    
    y_train = train[target]
    y_val = val[target]
    
    model = make_pipeline(
        FunctionTransformer(lambda X: X.to_dict(orient='records')),
        DictVectorizer(sparse=False),
        MinMaxScaler(),
        LogisticRegression(random_state=seed, solver='liblinear')
    )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]
    print("Classification Report for validation sets\n\n", classification_report(y_val, y_pred))
    print(f"ROC AUC for the validation set: {roc_auc_score(y_val, y_proba):.3f}\n")
    
    return model

def save_the_model(model, file_name="logreg"):
    """
    Save the machine learning model that contains preprocessor and regressor/classifier.
    
    Args:
    * model, .bin: model pipeline to predict unseen data later
    * file_name (default: "logreg"): name of the used classifier
    
    Output:
    * None
    
    """
    with open(f"../model/{file_name}.bin", 'wb') as file_out:
         dill.dump(model, file_out)
    
def load_the_model(path):
    """
    Load the trained model from directory
    
    Args:
    * path, str: trained model directory
    
    Output:
    * model, ... : a loaded model
    
    """
    with open(f"{path}", 'rb') as file_in:
         model = dill.load(file_in)
    
    return model

def experiment():
    # data preparation
    df = load_dataset("../dataset/telco-customer-churn.csv")
    df = parse_to_numeric(df, var='total_charges')
    df = df.fillna(0).drop('customer_id', axis=1)
    df['churn'] = df['churn'].map({"No": 0, "Yes": 1})
    
    # set up a validation framework
    full_train, test = split_dataset(df, target='churn', test_size=0.2)
    train, val = split_dataset(full_train, target='churn', test_size=0.25)
    
    # train the model
    model = train_the_model(train, val, target='churn')
    
    # save the model
    save_the_model(model, file_name="logreg")
    
    # load the model and use it to predict test set
    trained_model = load_the_model("../model/logreg.bin")
    
    X_test = test.drop("churn", axis=1)
    y_test = test["churn"]
    
    y_pred = trained_model.predict(X_test)
    y_proba = trained_model.predict_proba(X_test)[:, 1]
    print("Classification Report for test set\n\n", classification_report(y_test, y_pred))
    print(f"ROC AUC for the test set: {roc_auc_score(y_test, y_proba):.3f}\n")
    
    # sample unseen data to dictionary
    display(
        #X_test.sample().to_dict(orient='records'),
        #full_train.head()
    )
    
experiment()

Classification Report for validation sets

               precision    recall  f1-score   support

           0       0.84      0.90      0.87      1035
           1       0.65      0.52      0.58       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409

ROC AUC for the validation set: 0.848

Classification Report for test set

               precision    recall  f1-score   support

           0       0.85      0.90      0.87      1035
           1       0.66      0.55      0.60       374

    accuracy                           0.80      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.80      0.80      0.80      1409

ROC AUC for the test set: 0.851



### Load the dataset

In [2]:
df = load_dataset("../dataset/telco-customer-churn.csv")
df.head()

customer_id  gender  senior_citizen partner dependents  tenure  \
0  7590-VHVEG  Female               0     Yes         No       1   
1  5575-GNVDE    Male               0      No         No      34   
2  3668-QPYBK    Male               0      No         No       2   
3  7795-CFOCW    Male               0      No         No      45   
4  9237-HQITU  Female               0      No         No       2   

  phone_service    multiple_lines internet_service online_security  ...  \
0            No  No phone service              DSL              No  ...   
1           Yes                No              DSL             Yes  ...   
2           Yes                No              DSL             Yes  ...   
3            No  No phone service              DSL             Yes  ...   
4           Yes                No      Fiber optic              No  ...   

  device_protection tech_support streaming_tv streaming_movies  \
0                No           No           No               No   
1               Yes           No           No               No   
2                No           No           No               No   
3               Yes          Yes           No               No   
4                No           No           No               No   

         contract paperless_billing             payment_method  \
0  Month-to-month               Yes           Electronic check   
1        One year                No               Mailed check   
2  Month-to-month               Yes               Mailed check   
3        One year                No  Bank transfer (automatic)   
4  Month-to-month               Yes           Electronic check   

  monthly_charges  total_charges churn  
0           29.85          29.85    No  
1           56.95         1889.5    No  
2           53.85         108.15   Yes  
3           42.30        1840.75    No  
4           70.70         151.65   Yes  

[5 rows x 21 columns]

### Data Preparation

Monthly charges variable is a numerical variable but its data type is not either integer or float. We need to parse them into numeric.

In [3]:
df = parse_to_numeric(df, var='total_charges')
df['total_charges'].dtypes

dtype('float64')

### Set the validation framework

The assumption for this notebook:
* The dataset is splitted into three sets such as training, validation, and test sets with ratio 60:20:20 respectively.
* For EDA and build a model, I only use full train set (training and validation sets combined) to avoid leaking an information from test set to the analysis and model.

In [4]:
full_train, test = split_dataset(df, target='churn', test_size=0.2)
train, val = split_dataset(full_train, target='churn', test_size=0.25)

In [5]:
full_train.shape, val.shape, test.shape

((5634, 21), (1409, 21), (1409, 21))

### Target Variable Distribution

In [6]:
full_train['churn'].value_counts(normalize=True)

No     0.734647
Yes    0.265353
Name: churn, dtype: float64

### Dataset Summary

In [7]:
dataset_summary(full_train)

variable  no_unique pandas_dtype  missing_value  \
0         customer_id       5634       object              0   
1              gender          2       object              0   
2      senior_citizen          2        int64              0   
3             partner          2       object              0   
4          dependents          2       object              0   
5              tenure         73        int64              0   
6       phone_service          2       object              0   
7      multiple_lines          3       object              0   
8    internet_service          3       object              0   
9     online_security          3       object              0   
10      online_backup          3       object              0   
11  device_protection          3       object              0   
12       tech_support          3       object              0   
13       streaming_tv          3       object              0   
14   streaming_movies          3       object              0   
15           contract          3       object              0   
16  paperless_billing          2       object              0   
17     payment_method          4       object              0   
18    monthly_charges       1491      float64              0   
19      total_charges       5259      float64              9   
20              churn          2       object              0   

    %_missing_values                                       unique_value  
0           0.000000  [6061-GWWAV, 8464-EETCQ, 7621-VPNET, 6432-TWQL...  
1           0.000000                                     [Male, Female]  
2           0.000000                                             [0, 1]  
3           0.000000                                          [No, Yes]  
4           0.000000                                          [Yes, No]  
5           0.000000  [41, 57, 42, 5, 67, 1, 54, 21, 63, 62, 2, 13, ...  
6           0.000000                                          [Yes, No]  
7           0.000000                        [No, Yes, No phone service]  
8           0.000000                             [DSL, No, Fiber optic]  
9           0.000000                     [Yes, No internet service, No]  
10          0.000000                     [Yes, No internet service, No]  
11          0.000000                     [Yes, No internet service, No]  
12          0.000000                     [No, No internet service, Yes]  
13          0.000000                     [Yes, No internet service, No]  
14          0.000000                     [No, No internet service, Yes]  
15          0.000000               [One year, Two year, Month-to-month]  
16          0.000000                                          [No, Yes]  
17          0.000000  [Mailed check, Credit card (automatic), Electr...  
18          0.000000  [70.2, 18.8, 85.9, 85.4, 65.65, 44.55, 109.55,...  
19          0.159744  [2894.55, 1094.35, 3729.75, 401.1, 4322.85, 44...  
20          0.000000                                          [No, Yes]

### Modelling

In [11]:
def main():
    # data preparation
    df = load_dataset("../dataset/telco-customer-churn.csv")
    df = parse_to_numeric(df, var='total_charges')
    df = df.fillna(0)
    df['churn'] = df['churn'].map({"No": 0, "Yes": 1})
    
    # set up a validation framework
    full_train, test = split_dataset(df, target='churn', test_size=0.2)
    train, val = split_dataset(full_train, target='churn', test_size=0.25)
    
    # train the model
    model = train_the_model(train, val, target='churn')
    
    # save the model
    save_the_model(model, file_name="logreg")
    
    # load the model and use it to predict test set
    trained_model = load_the_model("../model/logreg.bin")
    
    X_test = test.drop("churn", axis=1)
    y_test = test["churn"]
    
    y_pred = trained_model.predict(X_test)
    y_proba = trained_model.predict_proba(X_test)[:, 1]
    print("Classification Report for test set\n\n", classification_report(y_test, y_pred))
    print(f"ROC AUC for the test set: {roc_auc_score(y_test, y_proba):.3f}\n")
    
main()

Classification Report for validation sets

               precision    recall  f1-score   support

           0       0.84      0.90      0.87      1035
           1       0.66      0.52      0.58       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409

ROC AUC for the validation set: 0.847

Classification Report for test set

               precision    recall  f1-score   support

           0       0.85      0.90      0.87      1035
           1       0.66      0.55      0.60       374

    accuracy                           0.81      1409
   macro avg       0.75      0.72      0.74      1409
weighted avg       0.80      0.81      0.80      1409

ROC AUC for the test set: 0.851

